In [1]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import Nystroem
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import MaxAbsScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from sklearn.preprocessing import FunctionTransformer
from copy import copy
from tpot import TPOTRegressor
from geopy.distance import geodesic

In [2]:
tpot_data = pd.read_csv('kc_house_data.csv')
zipcode = pd.read_excel('MedianZIP.xlsx')

# Data Preparation

In [3]:
tpot_data = tpot_data.drop(columns=['id','date'])

In [4]:
seattle_coordinates = (47.6062, -122.3321)
tpot_data['distance_from_seattle'] = tpot_data.apply(lambda x: geodesic(seattle_coordinates, (x['lat'], x['long'])).miles, axis=1)
tpot_data = tpot_data.join(pd.get_dummies(tpot_data['grade'], prefix='grade'))
tpot_data = tpot_data.join(pd.get_dummies(tpot_data['condition'], prefix='condition'))
tpot_data = tpot_data.join(pd.get_dummies(tpot_data['zipcode'], prefix='zipcode'))
tpot_data = tpot_data.drop(columns=['grade', 'zipcode', 'condition', 'lat', 'long'])
tpot_data = tpot_data.drop(columns=['sqft_living', 'sqft_living15', 'sqft_lot15'])
tpot_data.loc[tpot_data['yr_renovated'] != 0, 'yr_renovated'] = 1

In [5]:
features = tpot_data.drop(columns=['price']).values
X_train, X_test, y_train, y_test = train_test_split(features, tpot_data['price'], train_size=0.75, test_size=0.25)

# TPOT

In [6]:
tpot = TPOTRegressor(scoring='r2', generations=3, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))

Generation 1 - Current best internal CV score: 0.8722654830831151
Generation 2 - Current best internal CV score: 0.872448745717153
Generation 3 - Current best internal CV score: 0.872448745717153

Best pipeline: RandomForestRegressor(RidgeCV(input_matrix), bootstrap=False, max_features=0.7500000000000001, min_samples_leaf=12, min_samples_split=18, n_estimators=100)
0.8625076783799492


In [7]:
tpot.export('Housing.py')